In [15]:
# the objective of this notebook is to collect data from 2 APIs and store them as csv Files 
# The first API will be the Universalis API to collect 
# Historical market data in the consumable market in FFXIV
# The second API will be the XIVAPI to collect
# Item data and World data for the consumable market in FFXIV
# The data will be stored in csv files for further analysis

# Importing the required libraries
import requests
import json
import pandas as pd
import time
import datetime
import os
import numpy as np

# Universalis API no key required
# Setting the API Key for XIVAPI from the environment variable XIVAPI_KEY
xivapi_key = os.environ.get('XIVAPI_KEY')

# Setting the URL for the Universalis API
# https://docs.universalis.app/
univeralis_url = 'https://universalis.app/api/'

# Setting the URL for the XIVAPI
# https://xivapi.com/docs
xivapi_url = 'https://xivapi.com/'


In [16]:
#test cell to see if the API is working

import urllib.request, json

request = urllib.request.Request("http://xivapi.com/Item/1675")
request.add_header('User-Agent', '&lt;User-Agent&gt;')
data = json.loads(urllib.request.urlopen(request).read())

# I.Lv 80 Curtana
print ("I.Lv %d %s" % (data['LevelItem'], data['Name_en']))

I.Lv 80 Curtana


In [17]:
# lets start by getting the list of all the consumables in the game
# Consumables are items that can be used to provide a benefit to the player
# This includes food, potions, and other items that can be used to provide a
# temporary buff to the player
# The list of consumables will be used to collect the historical market data
# for each of the consumables
# The list of consumables will be collected from the XIVAPI
# The XIVAPI has a rate limit of 30 requests per second
# The category ID for consumables is 46
# The category ID for medicines is 44
# I can use the ItemKind.ID=5 parameter to get all items that are "Medicines & Meals" 

In [18]:
import urllib.request
import json
import pandas as pd

# Initialize an empty list to store all results
all_results = []

# Initialize page to 1
page = 1

while True:
    # Make the request
    request = urllib.request.Request(f"https://xivapi.com/search?filters=ItemKind.ID=5&limit=250&page={page}&private_key={xivapi_key}")
    request.add_header('User-Agent', '&lt;User-Agent&gt;')
    data = json.loads(urllib.request.urlopen(request).read())
    # The data is a dictionary with a key of "Results" and a value of a list of dictionaries

    # Add the results to our list
    all_results.extend(data['Results'])
    # The list of dictionaries contains the data for each of the consumables
    # The data for each consumable includes the name, the ID, and the category

    # Check if there are more pages
    if 'Pagination' in data and data['Pagination']['PageNext']:
        # If there are, increment the page counter
        page += 1
    else:
        # If there aren't, break the loop
        break

# Create a dataframe with all the results
consumables_df = pd.DataFrame(all_results)

# Display the first few rows of the dataframe
print(consumables_df)

# Save the data to a csv file
consumables_df.to_csv('..\\data\\consumables.csv', index=False)

         ID                  Icon                  Name          Url UrlType  \
0      4551  /i/020000/020601.png                Potion   /Item/4551    Item   
1      4552  /i/020000/020602.png             Hi-Potion   /Item/4552    Item   
2      4553  /i/020000/020603.png           Mega-Potion   /Item/4553    Item   
3      4554  /i/020000/020603.png              X-Potion   /Item/4554    Item   
4      4555  /i/020000/020621.png                 Ether   /Item/4555    Item   
...     ...                   ...                   ...          ...     ...   
2703  41411  /i/029000/029780.png     Lopoceras Elegans  /Item/41411    Item   
2704  41412  /i/028000/028969.png        Sidereal Whale  /Item/41412    Item   
2705  41413  /i/025000/025314.png    Prime Kukuru Beans  /Item/41413    Item   
2706  41414  /i/025000/025329.png  Sublime Kukuru Beans  /Item/41414    Item   
2707  41419  /i/029000/029586.png      Stargilt Lobster  /Item/41419    Item   

         _  _Score  
0     item       0

In [22]:
# Now that we have the list of consumables, we can use the Universalis API to collect the historical market data for each of the consumables
# The Universalis API has a rate limit of 25 requests per second
# The Universalis API provides historical market data for each of the servers in the game
# The historical market data includes the price and the quantity of the item

# First we need a nlist of all the servers in the game
# This will include Data Centers and Worlds
# The Data Centers are the larger groupings of servers based on location/region
# The Worlds are the individual servers that players can play on
# Because the Data Centers support cross-server play, we will group the data by Data Center

# https://universalis.app/api/v2/data-centers will return a list of all the data centers

data_centers = requests.get('https://universalis.app/api/v2/data-centers')


data_centers

<Response [200]>

In [23]:
data_centers = data_centers.json()
data_centers

[{'name': 'Elemental',
  'region': 'Japan',
  'worlds': [45, 49, 50, 58, 68, 72, 90, 94]},
 {'name': 'Gaia',
  'region': 'Japan',
  'worlds': [43, 46, 51, 59, 69, 76, 92, 98]},
 {'name': 'Mana',
  'region': 'Japan',
  'worlds': [23, 28, 44, 47, 48, 61, 70, 96]},
 {'name': 'Aether',
  'region': 'North-America',
  'worlds': [40, 54, 57, 63, 65, 73, 79, 99]},
 {'name': 'Primal',
  'region': 'North-America',
  'worlds': [35, 53, 55, 64, 77, 78, 93, 95]},
 {'name': 'Chaos',
  'region': 'Europe',
  'worlds': [39, 71, 80, 83, 85, 97, 400, 401]},
 {'name': 'Light',
  'region': 'Europe',
  'worlds': [33, 36, 42, 56, 66, 67, 402, 403]},
 {'name': 'Crystal',
  'region': 'North-America',
  'worlds': [34, 37, 41, 62, 74, 75, 81, 91]},
 {'name': 'Materia', 'region': 'Oceania', 'worlds': [21, 22, 86, 87, 88]},
 {'name': 'Meteor',
  'region': 'Japan',
  'worlds': [24, 29, 30, 31, 32, 52, 60, 82]},
 {'name': 'Dynamis',
  'region': 'North-America',
  'worlds': [404, 405, 406, 407]},
 {'name': 'NA Cloud 

In [24]:
#normalize the data
data_centers = pd.json_normalize(data_centers)
data_centers

,name,region,worlds
0,Elemental,Japan,"[45, 49, 50, 58, 68, 72, 90, 94]"
1,Gaia,Japan,"[43, 46, 51, 59, 69, 76, 92, 98]"
2,Mana,Japan,"[23, 28, 44, 47, 48, 61, 70, 96]"
3,Aether,North-America,"[40, 54, 57, 63, 65, 73, 79, 99]"
4,Primal,North-America,"[35, 53, 55, 64, 77, 78, 93, 95]"
5,Chaos,Europe,"[39, 71, 80, 83, 85, 97, 400, 401]"
6,Light,Europe,"[33, 36, 42, 56, 66, 67, 402, 403]"
7,Crystal,North-America,"[34, 37, 41, 62, 74, 75, 81, 91]"
8,Materia,Oceania,"[21, 22, 86, 87, 88]"
9,Meteor,Japan,"[24, 29, 30, 31, 32, 52, 60, 82]"


In [25]:
# Save the data to a csv file
data_centers.to_csv('..\\data\\data_centers.csv', index=False)

In [32]:
# Now to use https://universalis.app/api/v2/worlds to get a list of all the worlds
# This will return a list of all the worlds in the game
# This will include the World ID, and the World Name

worlds = requests.get('https://universalis.app/api/v2/worlds')
worlds

<Response [200]>

In [33]:
worlds = worlds.json()
worlds  

[{'id': 21, 'name': 'Ravana'},
 {'id': 22, 'name': 'Bismarck'},
 {'id': 23, 'name': 'Asura'},
 {'id': 24, 'name': 'Belias'},
 {'id': 28, 'name': 'Pandaemonium'},
 {'id': 29, 'name': 'Shinryu'},
 {'id': 30, 'name': 'Unicorn'},
 {'id': 31, 'name': 'Yojimbo'},
 {'id': 32, 'name': 'Zeromus'},
 {'id': 33, 'name': 'Twintania'},
 {'id': 34, 'name': 'Brynhildr'},
 {'id': 35, 'name': 'Famfrit'},
 {'id': 36, 'name': 'Lich'},
 {'id': 37, 'name': 'Mateus'},
 {'id': 39, 'name': 'Omega'},
 {'id': 40, 'name': 'Jenova'},
 {'id': 41, 'name': 'Zalera'},
 {'id': 42, 'name': 'Zodiark'},
 {'id': 43, 'name': 'Alexander'},
 {'id': 44, 'name': 'Anima'},
 {'id': 45, 'name': 'Carbuncle'},
 {'id': 46, 'name': 'Fenrir'},
 {'id': 47, 'name': 'Hades'},
 {'id': 48, 'name': 'Ixion'},
 {'id': 49, 'name': 'Kujata'},
 {'id': 50, 'name': 'Typhon'},
 {'id': 51, 'name': 'Ultima'},
 {'id': 52, 'name': 'Valefor'},
 {'id': 53, 'name': 'Exodus'},
 {'id': 54, 'name': 'Faerie'},
 {'id': 55, 'name': 'Lamia'},
 {'id': 56, 'name': 

In [34]:
#normalize the data
worlds = pd.json_normalize(worlds)
worlds

,id,name
0,21,Ravana
1,22,Bismarck
2,23,Asura
3,24,Belias
4,28,Pandaemonium
...,...,...
114,2075,카벙클
115,2076,초코보
116,2077,모그리
117,2078,톤베리


In [35]:
# Save the data to a csv file
worlds.to_csv('..\\data\\worlds.csv', index=False)

In [39]:
# Next we will get a list of all marketable items in the game https://universalis.app/api/v2/marketable
# This will return a list of all the itemIDs that are marketable

marketable = requests.get('https://universalis.app/api/v2/marketable')
marketable


<Response [200]>

In [40]:
marketable = marketable.json()
marketable

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 1601,
 1602,
 1603,
 1604,
 1605,
 1606,
 1607,
 1609,
 1611,
 1613,
 1614,
 1616,
 1621,
 1622,
 1625,
 1627,
 1633,
 1635,
 1636,
 1637,
 1639,
 1642,
 1643,
 1648,
 1649,
 1650,
 1657,
 1659,
 1662,
 1663,
 1666,
 1670,
 1673,
 1680,
 1681,
 1682,
 1683,
 1684,
 1685,
 1686,
 1688,
 1694,
 1697,
 1699,
 1701,
 1706,
 1708,
 1711,
 1716,
 1723,
 1728,
 1731,
 1732,
 1733,
 1736,
 1740,
 1743,
 1749,
 1750,
 1751,
 1752,
 1753,
 1754,
 1756,
 1758,
 1764,
 1766,
 1769,
 1771,
 1776,
 1778,
 1781,
 1786,
 1796,
 1799,
 1800,
 1801,
 1803,
 1806,
 1810,
 1813,
 1819,
 1820,
 1821,
 1822,
 1823,
 1824,
 1825,
 1827,
 1833,
 1836,
 1838,
 1840,
 1845,
 1847,
 1850,
 1855,
 1862,
 1867,
 1870,
 1871,
 1872,
 1875,
 1879,
 1882,
 1889,
 1891,
 1892,
 1893,
 1894,
 1895,
 1897,
 1899,
 1905,
 1908,
 1910,
 1915,
 1917,
 1920,
 1925,
 1932,
 1937,
 1940,
 1941,
 1942,
 1945,
 1949,
 1953,
 1958,
 1959,
 1960,
 

In [42]:
# no need to normalize this data as it is a list of itemIDs
marketable_df = pd.DataFrame(marketable)
marketable_df


,0
0,2
1,3
2,4
3,5
4,6
...,...
14532,41662
14533,41663
14534,41664
14535,41665


In [44]:
# save the data to a csv file
# with this list of marketable items we can filter our consumables list to only include items that are marketable
marketable_df.to_csv('..\\data\\marketable.csv', index=False)